In [1]:
# Import necessary libraries
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import Image
from time import sleep
import os
import numpy as np
from sklearn.metrics import classification_report

from amb_sdk.sdk import DarwinSdk

In [2]:
# Login
from login import username, password
ds = DarwinSdk()
ds.set_url('https://amb-demo-api.sparkcognition.com/v1/')
status, msg = ds.auth_login_user(username, password)

if not status:
    print(msg)
else:
    print('You are logged in!')

You are logged in!


In [3]:
dataset_name = 'district.csv'
df = pd.read_csv(dataset_name)
df.head()

,DISTNAME,DISTRICT,DZCNTYNM,REGION,DZRATING,DZCAMPUS,DPETALLC,DPETBLAP,DPETHISP,DPETWHIP,...,DPFPAVOCP,DPFPAGIFP,DPFPAATHP,DPFPAHSAP,DPFPREKP,DPFPAOTHP,DISTSIZE,COMMTYPE,PROPWLTH,TAXRATE
0,CAYUGA ISD,1902,001 ANDERSON,7,Met Standard,3,576,4.7,7.3,83.2,...,3.6,0.2,4.3,0.8,0,0,500 to 999,Rural,"$450,288 to < $512,741",Under $1.1659
1,ELKHART ISD,1903,001 ANDERSON,7,Met Standard,4,1267,5.3,10.8,79.8,...,5.6,0.1,5,1.1,0,0,"1,000 to 1,599",Non-metropolitan Stable,"$226,182 to < $235,915",$1.2781 to under $1.3998
2,FRANKSTON ISD,1904,001 ANDERSON,7,Met Standard,3,846,9.0,9.7,76.6,...,7.1,0.2,6.8,1.3,2.3,0.4,500 to 999,Rural,"$297,485 to < $318,066",$1.3998 and over
3,NECHES ISD,1906,001 ANDERSON,7,Met Standard,2,377,7.4,13.5,75.9,...,4.9,0.5,0,1.1,1.8,0,Under 500,Rural,"$297,485 to < $318,066",$1.2781 to under $1.3998
4,PALESTINE ISD,1907,001 ANDERSON,7,Met Standard,6,3453,27.5,40.0,28.2,...,6.8,0.2,6,0.6,2.1,1.5,"3,000 to 4,999",Independent Town,"$276,039 to < $297,485",$1.3998 and over


In [4]:
df = df[['DZRATING', 'DZCAMPUS','DPETALLC','DA0AT16R','DA0912DR16R','DAGC4X16R','DAGC5X15R','DAGC6X14R','DA0GR16N','DA0CT16R','DA0CC16R','DA0CSA16R','DA0CAA16R','DPSTTOSA','DPSTEXPA','DPFRAALLT','DPFRAALLK','DPFRASTAP','DZRVLOCP','DPFRAFEDP','DPFEAINST','DPFEAINSK','DISTSIZE','COMMTYPE', 'PROPWLTH', 'TAXRATE']]
df = df.rename(index=str, columns = {"DZRATING":"rating", "DZCAMPUS":"num_schools", "DPETALLC":"num_students", "DA0AT16R":"attendance_rate", "DA0912DR16R":"dropout_rate", "DAGC4X16R":"grad_rate_4_year", "DAGC5X15R":"grad_rate_5_year", "DAGC6X14R":"grad_rate_6_year", "DA0GR16N":"annual_grad_count", "DA0CT16R":"college_admissions_per_tested", "DA0CC16R":"college_admissions_at_crit", "DA0CSA16R":"average_sat", "DA0CAA16R":"average_act", "DPSTTOSA":"average_teacher_salary", "DPSTEXPA":"average_teacher_exp", "DPFRAALLT":"total_revenue", "DPFRAALLK":"total_revenue_per_pupil", "DPFRASTAP":"percent_revenue_from_state", "DZRVLOCP":"percent_revenue_from_local", "DPFRAFEDP":"percent_revenue_from_federal", "DPFEAINST":"instr_expenditures", "DPFEAINSK":"instr_expenditures_per_pupil", "DISTSIZE":"district_size", "COMMTYPE":"community_type", "PROPWLTH":"property_wealth", "TAXRATE":"tax_rate"})
df.head()

,rating,num_schools,num_students,attendance_rate,dropout_rate,grad_rate_4_year,grad_rate_5_year,grad_rate_6_year,annual_grad_count,college_admissions_per_tested,...,total_revenue_per_pupil,percent_revenue_from_state,percent_revenue_from_local,percent_revenue_from_federal,instr_expenditures,instr_expenditures_per_pupil,district_size,community_type,property_wealth,tax_rate
0,Met Standard,3,576,96.8,1,93.5,96.6,89.2,45,60,...,15318,40.4,38.6,21,4750155,8363,500 to 999,Rural,"$450,288 to < $512,741",Under $1.1659
1,Met Standard,4,1267,96.6,0.3,98.6,98.6,98.7,74,56.8,...,9887,61.2,32.9,5.9,6535726,5254,"1,000 to 1,599",Non-metropolitan Stable,"$226,182 to < $235,915",$1.2781 to under $1.3998
2,Met Standard,3,846,96.2,0.4,100,100,100,50,38,...,11153,50.7,43.2,6.1,4099594,4875,500 to 999,Rural,"$297,485 to < $318,066",$1.3998 and over
3,Met Standard,2,377,96.4,0,94.3,96.9,97.6,35,60,...,10892,56.7,36.3,7.1,2190799,5720,Under 500,Rural,"$297,485 to < $318,066",$1.2781 to under $1.3998
4,Met Standard,6,3453,95.7,0.2,89.2,96.8,97.8,178,32,...,12311,43.4,42.8,13.8,18198163,5376,"3,000 to 4,999",Independent Town,"$276,039 to < $297,485",$1.3998 and over


In [6]:
df[0:len(df)//5].to_csv('test_data.csv')
df[len(df)//5:].to_csv('train_data.csv')